# mRNA something title
Authors: Siona Tagare, Shannen Espinosa, Gun Suer

YQuantum 2024 April 13-14

This is a ...

In [16]:
import numpy as np
import math

from CAI import CAI
from Bio.Seq import Seq

from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Estimator, Sampler, Session, Options
from qiskit_aer import AerSimulator

In [3]:
p = "MSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPY"

In [4]:
def p2codons(p):
    aa_codons = {
        'A': ['GCT', 'GCC', 'GCA', 'GCG'],
        'R': ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'],
        'N': ['AAT', 'AAC'],
        'D': ['GAT', 'GAC'],
        'C': ['TGT', 'TGC'],
        'Q': ['CAA', 'CAG'],
        'E': ['GAA', 'GAG'],
        'G': ['GGT', 'GGC', 'GGA', 'GGG'],
        'H': ['CAT', 'CAC'],
        'I': ['ATT', 'ATC', 'ATA'],
        'L': ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'],
        'K': ['AAA', 'AAG'],
        'M': ['ATG'],
        'F': ['TTT', 'TTC'],
        'P': ['CCT', 'CCC', 'CCA', 'CCG'],
        'S': ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'],
        'T': ['ACT', 'ACC', 'ACA', 'ACG'],
        'W': ['TGG'],
        'Y': ['TAT', 'TAC'],
        'V': ['GTT', 'GTC', 'GTA', 'GTG'],
    }
    
    codon_seq = []
    for aa in p:
        if aa in aa_codons:
            codons = aa_codons[aa]
            codon_seq.append(codons)
        else:
            codon_seq.append([''])

    return codon_seq

codons = p2codons(p)
print(codons)

[['ATG'], ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], ['AAT', 'AAC'], ['AAT', 'AAC'], ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], ['GAT', 'GAC'], ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], ['AAA', 'AAG'], ['GTT', 'GTC', 'GTA', 'GTG'], ['GGT', 'GGC', 'GGA', 'GGG'], ['GGT', 'GGC', 'GGA', 'GGG'], ['AAT', 'AAC'], ['TAT', 'TAC'], ['AAT', 'AAC'], ['TAT', 'TAC'], ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], ['TAT', 'TAC'], ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], ['TTT', 'TTC'], ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], ['AAA', 'AAG'], ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], ['AAT', 'AAC'], ['TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG'], ['AAA', 'AAG'], ['CCT', 'CCC', 'CCA', 'CCG'], ['TTT', 'TTC'], ['GAA', 'GAG'], ['CGT', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], ['GAT', 'GAC'], ['ATT', 'ATC', 'ATA'], ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC'], ['ACT', 'ACC', 'ACA', 'ACG'], ['GAA', 'GAG'], ['ATT', 'ATC', 'ATA'], ['TAT', 'TAC'], ['CAA', 'CAG'], ['GCT'

In [48]:
def probabilities(codon):
    prob = {
        'TTT': 0.176, 'TTC': 0.203,
        'TTA': 0.077, 'TTG': 0.129,
        'TCT': 0.152, 'TCC': 0.177,
        'TCA': 0.122, 'TCG': 0.044,
        'TAT': 0.122, 'TAC': 0.153,
        'TAA': 0.01, 'TAG': 0.008,
        'TGT': 0.106, 'TGC': 0.126,
        'TGA': 0.016, 'TGG': 0.132,
        'CTT': 0.132, 'CTC': 0.196,
        'CTA': 0.072, 'CTG': 0.396,
        'CCT': 0.175, 'CCC': 0.198,
        'CCA': 0.169, 'CCG': 0.069,
        'CAT': 0.109, 'CAC': 0.151,
        'CAA': 0.123, 'CAG': 0.342,
        'CGT': 0.045, 'CGC': 0.104,
        'CGA': 0.062, 'CGG': 0.114,
        'ATT': 0.16, 'ATC': 0.208,
        'ATA': 0.075, 'ATG': 0.22,
        'ACT': 0.131, 'ACC': 0.189,
        'ACA': 0.151, 'ACG': 0.061,
        'AAT': 0.17, 'AAC': 0.191,
        'AAA': 0.244, 'AAG': 0.319,
        'AGT': 0.121, 'AGC': 0.195,
        'AGA': 0.122, 'AGG': 0.12,
        'GTT': 0.11, 'GTC': 0.145,
        'GTA': 0.071, 'GTG': 0.281,
        'GCT': 0.184, 'GCC': 0.277,
        'GCA': 0.158, 'GCG': 0.074,
        'GAT': 0.218, 'GAC': 0.251,
        'GAA': 0.29, 'GAG': 0.396,
        'GGT': 0.108, 'GGC': 0.222,
        'GGA': 0.165, 'GGG': 0.165
    }
    return prob.get(codon, 0)

print(probabilities('GCT'))
print(probabilities('TAG'))

0.184
0.008


In [6]:
def gc_content(codon):
    gc_count = 0.5
    total_count = 0
    for c in codon:
        gc_count += codon.count('G') + codon.count('C')
        total_count += len(codon)
    gc_content = gc_count / total_count if total_count > 0 else 0
    return gc_content
    
print(gc_content("GGG"))
print(gc_content("ACC"))
print(gc_content("AAA"))

1.0555555555555556
0.7222222222222222
0.05555555555555555


In [7]:
def minimize_repeats(codon1, codon2):
    repeat_count = 0

    if codon1[-1] == codon2[0]:
        repeat_count += 2
        if codon1[1] == codon1[-1] and codon1[0] == codon1[-1]:
            repeat_count += 2
        elif codon1[1] == codon1[-1]:
            repeat_count += 1

        if codon2[0] == codon2[1] and codon2[0] == codon2[-1]:
            repeat_count += 2
        elif codon2[0] == codon2[1]:
            repeat_count += 1

    if repeat_count == 0:
        return 1
    else:
        return 1 / repeat_count

print(minimize_repeats("ATT", "ACA"))
print(minimize_repeats("ATT", "TCA"))
print(minimize_repeats("ATT", "TTT"))
print(minimize_repeats("TTT", "TTA"))
print(minimize_repeats("TTT", "TTT"))

1
0.3333333333333333
0.2
0.2
0.16666666666666666


In [8]:
def cai_value(codon1, codon2):
    cai = CAI(Seq(codon1), reference=[Seq(codon2)])
    if math.isnan(cai):
        return 0
    else:
        return cai

print(cai_value("ATA", "ATG"))
print(cai_value("TAG", "GAT"))
print(cai_value("AAT", "AAC"))
print(cai_value("AAC", "ATA"))
print(cai_value("GCG", "AAA"))

1.0
0
0.5
1.0
1.0


/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [9]:
def score(codon1, codon2):
    score = probabilities(codon1) * probabilities(codon2) * gc_content(codon1) * gc_content(codon2) * minimize_repeats(codon1, codon2) * cai_value(codon1, codon2)
    return score * 1000

print(score("ATA", "ATG"))
print(score("TAG", "GAT"))
print(score("AAT", "AAC"))
print(score("AAC", "ATA"))
print(score("GCG", "AAA"))

0.015194537037037038
0.0
0.019696033950617288
0.05452157407407408
0.27111064814814817


In [10]:
def best_score(codonlist1, codonlist2):
    max_score = 0
    
    for codon1 in codonlist1:
        for codon2 in codonlist2:
            current_score = score(codon1, codon2)
            if current_score > max_score:
                max_score = current_score
    
    return max_score

codonlist1 = ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC']
codonlist2 = ['TCT', 'TCC', 'TCA', 'TCG', 'AGT', 'AGC']

best = best_score(codonlist1, codonlist2)
print("Best score:", best)

Best score: 2.5122371604938274


In [11]:
j_matrix = np.zeros((len(p), len(p)))
codons = p2codons(p)

def calculate_j_matrix():
    for i in range(len(p) - 1):
        j_matrix[i, i + 1] = best_score(codons[i], codons[i + 1])
        j_matrix[i + 1, i] = j_matrix[i, i + 1]
    
    return j_matrix
        
print(calculate_j_matrix())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.83230735 ... 0.         0.         0.        ]
 [0.         0.83230735 0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         1.40513269 0.        ]
 [0.         0.         0.         ... 1.40513269 0.         0.59874219]
 [0.         0.         0.         ... 0.         0.59874219 0.        ]]


In [12]:
j_matrix = calculate_j_matrix()
binary_vars = []

qp = QuadraticProgram()
linear_coefficients = {}
quadratic_coefficients = {}
for i in range(len(j_matrix)):
    for j in range(len(j_matrix)):
        var = qp.binary_var(name=f'x_{i}_{j}')
        binary_vars.append(var)
        linear_coefficients[f'x_{i}_{j}'] = j_matrix[i, j]
        if j > i:
            quadratic_coefficients[(f'x_{i}_{j-1}', f'x_{i+1}_{j}')] = j_matrix[i, j]
            
for j in range(len(j_matrix)):
    constraint_dict = {var_name: 1.0 for var_name in [f'x_{i}_{j}' for i in range(len(j_matrix))]}
    qp.linear_constraint(linear=constraint_dict, sense='==', rhs=1.0)

qp.maximize(linear=linear_coefficients, quadratic=quadratic_coefficients)

In [17]:
service = QiskitRuntimeService(channel="ibm_quantum")
backend = service.get_backend("ibmq_qasm_simulator") # delete line when ready
simulator = AerSimulator()

qiskit_runtime_service.__init__:INFO:2024-04-14 12:42:05,970: Default instance: ibm-q/open/main


In [ ]:
exact_mes = NumPyMinimumEigensolver()
exact = MinimumEigenOptimizer(exact_mes)
exact_result = exact.solve(qp)
print("classical result")
print_results(exact_result)

In [ ]:
# with Session(service=service, backend=backend) as session:
#    qaoa = QAOA(sampler=Sampler(session=session), optimizer=COBYLA())
#    min_eigen_optimizer = MinimumEigenOptimizer(qaoa)
# quantum_result = min_eigen_optimizer.solve(qp)

qaoa = QAOA(sampler=simulator, optimizer=COBYLA())
min_eigen_optimizer = MinimumEigenOptimizer(qaoa)
quantum_result = min_eigen_optimizer.solve(qp)
print("quantum results:")
print_results(quantum_result)

In [ ]:
print(linear_coefficients)
print(quadratic_coefficients)